## Hidden Markov Model

Hidden Markov Model은 관측 시퀀스에 대한 분포를 표현하는 확률적 모델이다. HMM은 가장 단순한 Dynamic Bayesian Network이고, 금융을 포함한 많은 응용 분야에서 강력한 모델임이 입증됐다. 이제까지는 주로 IID 관측만을 가정했다. 물론, 금융 모델링은 많은 경우 관측이 이전 관측, 그리고 그 이전 관측에만 의존하는 마르코프 설정에서 수행된다. HMM은 확률적 모델링의 중요한 아이디어를 함축하고 있으므로 여기서 간단히 살펴본다. 특히 이들은 은닉 변수와 전환(switching)을 이해하는 데 통찰력을 제공한다. Gating과 같은 알고리즘을 사용하는 RNN, LSTM과 같은 동적 순환 신경망은 암묵적인 모델링 단계임으로, 금융의 국면 전환에 필요한 만큼 명시적으로 제어할 수 없다.

이제 시점 $t$에 이산 상태 $s_t$가 관측자에게 보이지 않는다고 가정하자. 이 설정은 은닉 변수가 IID라고 가정하는 Mixtual Model과는 다르다. 시점 $t$ 관측 $y_t$는 다른 모든 시점의 상태와 독립이라고 가정한다. Markov 특성에 의해 상태 시퀀스 $s := \{s_t\}_{i=1}^T$와 관측 시퀀스 $y = \{y_t\}_{t=1}^T$의 결합 확률은 전이 확률 밀도 $p(s_t|s_{t-1})$과 출력 확률 밀도 $p(y_t|s_t)$의 곱으로 구할 수 있다.

$$p(s, y) = p(s_1)p(y_1 | s_1) \prod_{t=2}^T p(s_t | s_{t-1})p(y_t | s_t)$$

우선 모든 상태 $s_t \in \{1, \dots, K\}$와 모든 시점에 대해 소위 전방과 후방 확률을 각각 도입한다.

$$F_t(s) := \mathbb{P}(s_t = s, y_{1:t}), B_t(s) := p(y_{t_1:T} | s_t = s)$$

여기서 관항에 따라 $B_T(s) = 1$이다. 모든 $t \in \{1, \dots, T\}$와 모든 $r, s \in \{1, \dots, K\}$에 대해 다음을 얻는다.

$$\mathbb{P}(s_t = s, y) = F_t(s)B_t(s)$$

그리고 전방과 후방 확률을 결합하면 다음을 산출할 수 있다.

$$\mathbb{P}(s_{t-1} = r, s_t = s, y) = F_{t-1}(r) \mathbb{P}(s_t = s|s_{t-1} = r)p(y_t | s_t = s)B_t(s)$$

Baum-Welch Algorithm으로도 알려져 있는 이 forward-backward algorithm은 EM algorithm 클래스에 속하는 HMM을 적합화하기 위한 비지도학습 알고리즘이다.

### 1. Viterbi Algorithm

특정 은닉 상태의 실현 확률을 찾는 것에 더해 가장 가능한 시퀀스 실현을 찾을 수 있다. 이 시퀀스는 Viterbi algorithm을 사용해 추정할 수 있다. 다시 $T$개의 관측 시퀀스를 관측한다고 가정하자.

$$\mathbf{y} = \{y_1, \dots, y_T\}$$

그러나 각 $1 \leq t \leq T, y_t \in O$에 대해 $O = \{o_1, o_2, \dots, o_N\}, N \in \mathbb{N}$은 이제 어떤 관측 공간에 있다. 각 $1 \leq t \leq T$에 대해 $y_t$는 (은닉)상태 $s_t \in \mathcal{S}$에 의해 주도되며, 여기서 $\mathcal{S} := \{ \int_1, \dots, \int_K\}, K \in \mathbb{N}$은 어떤 상태 공간이다. 예를 들어 $y_t$는 어떤 회사채의 신용 등급이고 $s_t$는 관련 산업 섹터의 전체적인 건전성과 같은 어떤 잠재 변수를 가리킬 수 있다.

주어진 $\mathbf{y}$에 대해 가장 가능한 은닉 상태의 시퀀스는 무엇일까?

$$\mathbf{x} = \{x_1, x_2,\ dots, x_T\}$$

이 문제에 답하고자 추가적인 변수들을 도입한다. 첫째, 반드시 다음과 같은 초기 확률 집합이 주어져야만 한다.

$$\mathcal{\pi} = \{\pi_1, \dots, \pi_K\}$$

따라서 $\pi_i $는 $s_1 = \int_i, 1 \leq i \leq K$일 확률이다. 또한 원소 $A_{ij}, 1 \leq i, j \leq K$가 상태 $\int_i$에서 상태 $\int_j$로 전이하는 전이 확률인 전이 행렬(transition matrix) $A \in \mathbb{R}^{K \times K}$을 설정할 필요가 있다. 마지막으로 원소 $B_{ij}, 1 \leq i \leq K, 1 \leq j \leq N$가 상태 $\int_i$에서 $o_j$를 관측하는 확률인 출력 행렬(emission matrix) $B \in \mathbb{R}^{K \times N}$을 필요로 한다.

관측 시퀀스 $y = \{y_1, \dots, y_T\}$를 산출하는 가장 가능한 상태 시퀀스 $s$는 다음의 재귀 관계를 만족한다.

$$
\begin{align}
V_{1,k} = & \mathbb{P}(y_1 | s_1 = \int_k)\cdot \pi_k \\ 
V_{t,k} = & \max_{1 \leq i \leq K} \left( \mathbb{P}\left(y_t | s_t = \int_k\right) \cdot A_{ik} \cdot V_{t-1, i}\right)
\end{align}
$$

여기서 $V_{t,k}$는 $s_t = \int_k$일 때의 가장 가능한 상태 시퀀스 $\{s_1, \dots, s_t\}$의 확률이다.

$$V_{t,k} = \mathbb{P}\left(s_1, \dots, s_t, y_1, \dots, y_t| s_t = \int_k\right)$$